In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import matplotlib
from numpy import random
import copy
import scipy

In [ ]:
# for the font of the plots
matplotlib.rc("font", **{"family":"serif","serif":["Computer Modern Roman"]})
matplotlib.rc("text", usetex=True)

## Toy model data

In [ ]:
# simulation parameters
numx = 30
numy = 30
N = numx*numy
R = 0.5
nrec = 1000
n_j = int(nrec*0.8)
wnum = int(n_j/2)+1

# toy model
spacing_tm = 1.0
dx_tm = 0.5*spacing_tm
dy_tm = 0.5*spacing_tm
Lx_tm = spacing_tm * 2 * R * numx
Ly_tm = 0.5 * np.sqrt(3) * numy * spacing_tm * 2 * R
xnum_tm = round(Lx_tm/dx_tm)
ynum_tm = round(Ly_tm/dy_tm)
dkx_tm = 2*np.pi/Lx_tm
dky_tm = 2*np.pi/Ly_tm
dt_tm = 0.4
dw_tm = 2*np.pi/(dt_tm*n_j)

# frequency domain 
kxrange_tm = np.zeros(xnum_tm)
kyrange_tm = np.zeros(ynum_tm)
wrange_tm = dw_tm*np.arange(wnum)

for i in range(xnum_tm):
    kxrange_tm[i] = -np.pi/dx_tm + dkx_tm*i
for j in range(ynum_tm):
    kyrange_tm[j] = -np.pi/dy_tm + dky_tm*j
    
kx_tm,ky_tm = np.meshgrid(kxrange_tm,kyrange_tm,indexing='ij')

# load current correlation files
# toy model

with open('Cll_toy.dat','rb') as f:
    Cll_tm = np.fromfile(f,np.float64)
Cll_tm = Cll_tm.reshape((xnum_tm,ynum_tm,wnum))

with open('Cltr_toy.dat','rb') as f:
    Cltr_tm = np.fromfile(f,np.float64)
Cltr_tm = Cltr_tm.reshape((xnum_tm,ynum_tm,wnum))

with open('Clti_toy.dat','rb') as f:
    Clti_tm = np.fromfile(f,np.float64)
Clti_tm = Clti_tm.reshape((xnum_tm,ynum_tm,wnum))

with open('Ctt_toy.dat','rb') as f:
    Ctt_tm = np.fromfile(f,np.float64)
Ctt_tm = Ctt_tm.reshape((xnum_tm,ynum_tm,wnum))


## Starfish embryo model data

In [ ]:
# simulation parameters
# starfish model

numx = 30
numy = 30
N = numx*numy
R = 0.5
nrec = 1000
n_j = int(nrec*0.8)
wnum = int(n_j/2)+1

spacing_sm = 1.2
dx_sm = 0.5*spacing_sm
dy_sm = 0.5*spacing_sm
Lx_sm = spacing_sm * 2 * R * numx
Ly_sm = 0.5 * np.sqrt(3) * numy * spacing_sm * 2 * R
xnum_sm = round(Lx_sm/dx_sm)
ynum_sm = round(Ly_sm/dy_sm)
dkx_sm = 2*np.pi/Lx_sm
dky_sm = 2*np.pi/Ly_sm
dt_sm = 0.1
dw_sm = 2*np.pi/(dt_sm*n_j)

# frequency domain 
kxrange_sm = np.zeros(xnum_sm)
kyrange_sm = np.zeros(ynum_sm)
wrange_sm = dw_sm*np.arange(wnum)

for i in range(xnum_sm):
    kxrange_sm[i] = -np.pi/dx_sm + dkx_sm*i
for j in range(ynum_sm):
    kyrange_sm[j] = -np.pi/dy_sm + dky_sm*j
    
kx_sm,ky_sm = np.meshgrid(kxrange_sm,kyrange_sm,indexing='ij')

# load current correlation files
# starfish model

with open('Cll_starfish.dat','rb') as f:
    Cll_sm = np.fromfile(f,np.float64)
Cll_sm = Cll_sm.reshape((xnum_sm,ynum_sm,wnum))

with open('Cltr_starfish.dat','rb') as f:
    Cltr_sm = np.fromfile(f,np.float64)
Cltr_sm = Cltr_sm.reshape((xnum_sm,ynum_sm,wnum))

with open('Clti_starfish.dat','rb') as f:
    Clti_sm = np.fromfile(f,np.float64)
Clti_sm = Clti_sm.reshape((xnum_sm,ynum_sm,wnum))

with open('Ctt_starfish.dat','rb') as f:
    Ctt_sm = np.fromfile(f,np.float64)
Ctt_sm = Ctt_sm.reshape((xnum_sm,ynum_sm,wnum))

## Path in the first Brillouin zone

In [ ]:
# find the path Gamma -> K -> M -> Gamma
# indices for Gamma, K, M points in the first BZ

Gpoint = np.array([30,26],dtype=int)
Mpoint = np.array([45,33],dtype=int)
Kpoint = np.array([40,41],dtype=int)

# indices for the path G->K->M->G

idisGK = Kpoint[0]-Gpoint[0]
jdisGK = Kpoint[1]-Gpoint[1]
numGtoK = max(np.abs(idisGK),np.abs(jdisGK))
GtoK = np.zeros([2,numGtoK],dtype = int)
GtoK[0,0]=Gpoint[0]
GtoK[1,0]=Gpoint[1]
if np.abs(idisGK)>np.abs(jdisGK):
    idisGKsign = idisGK/np.abs(idisGK)
    for i in range(1,numGtoK):
        GtoK[0,i]=GtoK[0,0]+i*idisGKsign
        GtoK[1,i]=int(GtoK[1,0]+i*idisGKsign*jdisGK/idisGK)
else:
    jdisGKsign = jdisGK/np.abs(jdisGK)
    for i in range(1,numGtoK):
        GtoK[1,i]=GtoK[1,0]+i*jdisGKsign
        GtoK[0,i]=int(GtoK[0,0]+i*jdisGKsign*idisGK/jdisGK)

idisKM = Mpoint[0]-Kpoint[0]
jdisKM = Mpoint[1]-Kpoint[1]
numKtoM = max(np.abs(idisKM),np.abs(jdisKM))
KtoM = np.zeros([2,numKtoM],dtype = int)
KtoM[0,0]=Kpoint[0]
KtoM[1,0]=Kpoint[1]
if np.abs(idisKM)>np.abs(jdisKM):
    idisKMsign = idisKM/np.abs(idisKM)
    for i in range(1,numKtoM):
        KtoM[0,i]=KtoM[0,0]+i*idisKMsign
        KtoM[1,i]=int(KtoM[1,0]+i*idisKMsign*jdisKM/idisKM)
else:
    jdisKMsign = jdisKM/np.abs(jdisKM)
    for i in range(1,numKtoM):
        KtoM[1,i]=KtoM[1,0]+i*jdisKMsign
        KtoM[0,i]=int(KtoM[0,0]+i*jdisKMsign*idisKM/jdisKM)
        
idisMG = Gpoint[0]-Mpoint[0]
jdisMG = Gpoint[1]-Mpoint[1]
numMtoG = max(np.abs(idisMG),np.abs(jdisMG))
MtoG = np.zeros([2,numMtoG+1],dtype = int)
MtoG[0,0]=Mpoint[0]
MtoG[1,0]=Mpoint[1]
if np.abs(idisMG)>np.abs(jdisMG):
    idisMGsign = idisMG/np.abs(idisMG)
    for i in range(1,numMtoG):
        MtoG[0,i]=MtoG[0,0]+i*idisMGsign
        MtoG[1,i]=int(MtoG[1,0]+i*idisMGsign*jdisMG/idisMG)
else:
    jdisMGsign = jdisMG/np.abs(jdisMG)
    for i in range(1,numMtoG):
        MtoG[1,i]=MtoG[1,0]+i*jdisMGsign
        MtoG[0,i]=int(MtoG[0,0]+i*jdisMGsign*idisMG/jdisMG)
MtoG[0,-1]=Gpoint[0]
MtoG[1,-1]=Gpoint[1]

### toy model

In [ ]:
# along the path in the first Brillouin zone Gamma -> K -> M -> Gamma
Clltm_GtoK = copy.copy(Cll_tm)
Clltm_GtoK = Clltm_GtoK[GtoK[0,:],GtoK[1,:],:]
Clltm_KtoM = copy.copy(Cll_tm)
Clltm_KtoM = Clltm_KtoM[KtoM[0,:],KtoM[1,:],:]
Clltm_MtoG = copy.copy(Cll_tm)
Clltm_MtoG = Clltm_MtoG[MtoG[0,:],MtoG[1,:],:]
Clltm_GKMG = np.concatenate((Clltm_GtoK,Clltm_KtoM,Clltm_MtoG),axis=0)

Cltrtm_GtoK = copy.copy(Cltr_tm)
Cltrtm_GtoK = Cltrtm_GtoK[GtoK[0,:],GtoK[1,:],:]
Cltrtm_KtoM = copy.copy(Cltr_tm)
Cltrtm_KtoM = Cltrtm_KtoM[KtoM[0,:],KtoM[1,:],:]
Cltrtm_MtoG = copy.copy(Cltr_tm)
Cltrtm_MtoG = Cltrtm_MtoG[MtoG[0,:],MtoG[1,:],:]
Cltrtm_GKMG = np.concatenate((Cltrtm_GtoK,Cltrtm_KtoM,Cltrtm_MtoG),axis=0)

Cltitm_GtoK = copy.copy(Clti_tm)
Cltitm_GtoK = Cltitm_GtoK[GtoK[0,:],GtoK[1,:],:]
Cltitm_KtoM = copy.copy(Clti_tm)
Cltitm_KtoM = Cltitm_KtoM[KtoM[0,:],KtoM[1,:],:]
Cltitm_MtoG = copy.copy(Clti_tm)
Cltitm_MtoG = Cltitm_MtoG[MtoG[0,:],MtoG[1,:],:]
Cltitm_GKMG = np.concatenate((Cltitm_GtoK,Cltitm_KtoM,Cltitm_MtoG),axis=0)

Ctttm_GtoK = copy.copy(Ctt_tm)
Ctttm_GtoK = Ctttm_GtoK[GtoK[0,:],GtoK[1,:],:]
Ctttm_KtoM = copy.copy(Ctt_tm)
Ctttm_KtoM = Ctttm_KtoM[KtoM[0,:],KtoM[1,:],:]
Ctttm_MtoG = copy.copy(Ctt_tm)
Ctttm_MtoG = Ctttm_MtoG[MtoG[0,:],MtoG[1,:],:]
Ctttm_GKMG = np.concatenate((Ctttm_GtoK,Ctttm_KtoM,Ctttm_MtoG),axis=0)

qnum_tm = np.size(Clltm_GKMG,axis=0)
qtm_band,wtm_band = np.meshgrid(np.arange(qnum_tm),wrange_tm,indexing='ij')

fromthisw = 1 # not include w=0
uptothisw = -1 # high frequency w is not relevant
wtm_band = wtm_band[:,fromthisw:uptothisw] # only take w>0.
qtm_band = qtm_band[:,fromthisw:uptothisw]
Clltm_band = Clltm_GKMG[:,fromthisw:uptothisw]
Cltrtm_band = Cltrtm_GKMG[:,fromthisw:uptothisw]
Cltitm_band = Cltitm_GKMG[:,fromthisw:uptothisw]
Ctttm_band = Ctttm_GKMG[:,fromthisw:uptothisw]


### Starfish embryo model

In [ ]:
# along the path in the first Brillouin zone Gamma -> K -> M -> Gamma
Cllsm_GtoK = copy.copy(Cll_sm)
Cllsm_GtoK = Cllsm_GtoK[GtoK[0,:],GtoK[1,:],:]
Cllsm_KtoM = copy.copy(Cll_sm)
Cllsm_KtoM = Cllsm_KtoM[KtoM[0,:],KtoM[1,:],:]
Cllsm_MtoG = copy.copy(Cll_sm)
Cllsm_MtoG = Cllsm_MtoG[MtoG[0,:],MtoG[1,:],:]
Cllsm_GKMG = np.concatenate((Cllsm_GtoK,Cllsm_KtoM,Cllsm_MtoG),axis=0)

Cltrsm_GtoK = copy.copy(Cltr_sm)
Cltrsm_GtoK = Cltrsm_GtoK[GtoK[0,:],GtoK[1,:],:]
Cltrsm_KtoM = copy.copy(Cltr_sm)
Cltrsm_KtoM = Cltrsm_KtoM[KtoM[0,:],KtoM[1,:],:]
Cltrsm_MtoG = copy.copy(Cltr_sm)
Cltrsm_MtoG = Cltrsm_MtoG[MtoG[0,:],MtoG[1,:],:]
Cltrsm_GKMG = np.concatenate((Cltrsm_GtoK,Cltrsm_KtoM,Cltrsm_MtoG),axis=0)

Cltism_GtoK = copy.copy(Clti_sm)
Cltism_GtoK = Cltism_GtoK[GtoK[0,:],GtoK[1,:],:]
Cltism_KtoM = copy.copy(Clti_sm)
Cltism_KtoM = Cltism_KtoM[KtoM[0,:],KtoM[1,:],:]
Cltism_MtoG = copy.copy(Clti_sm)
Cltism_MtoG = Cltism_MtoG[MtoG[0,:],MtoG[1,:],:]
Cltism_GKMG = np.concatenate((Cltism_GtoK,Cltism_KtoM,Cltism_MtoG),axis=0)

Cttsm_GtoK = copy.copy(Ctt_sm)
Cttsm_GtoK = Cttsm_GtoK[GtoK[0,:],GtoK[1,:],:]
Cttsm_KtoM = copy.copy(Ctt_sm)
Cttsm_KtoM = Cttsm_KtoM[KtoM[0,:],KtoM[1,:],:]
Cttsm_MtoG = copy.copy(Ctt_sm)
Cttsm_MtoG = Cttsm_MtoG[MtoG[0,:],MtoG[1,:],:]
Cttsm_GKMG = np.concatenate((Cttsm_GtoK,Cttsm_KtoM,Cttsm_MtoG),axis=0)

qnum_sm = np.size(Cllsm_GKMG,axis=0)
qsm_band,wsm_band = np.meshgrid(np.arange(qnum_sm),wrange_sm,indexing='ij')

fromthisw = 1 # not include w=0
uptothisw = -1 # high frequency w is not relevant
wsm_band = wsm_band[:,fromthisw:uptothisw] # only take w>0.
qsm_band = qsm_band[:,fromthisw:uptothisw]
Cllsm_band = Cllsm_GKMG[:,fromthisw:uptothisw]
Cltrsm_band = Cltrsm_GKMG[:,fromthisw:uptothisw]
Cltism_band = Cltism_GKMG[:,fromthisw:uptothisw]
Cttsm_band = Cttsm_GKMG[:,fromthisw:uptothisw]


## Fitting

In [ ]:
# skewed Cauchy distribution
def skewed_Cauchy(x,x0,a,b,s):
    y = (a/b)*(1/(1+(x-x0)**2/((b**2)*(1+s*np.sign(x-x0))**2)))
    return y

### Toy model

In [ ]:
# fit each Cw along the path to get the max w at each q on the path
wmax_est = 8

# longitudinal 
qlltm_path = np.size(Clltm_band,0)-3
wtm_range = wtm_band[0,:]
wlltmmax_path = np.zeros(qlltm_path)
wlltmerr_path = np.zeros(qlltm_path)

for i in range(qlltm_path):
    thisClltm = Clltm_band[i+1,:]
    popt_Clltm,pcov_Clltm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wtm_range,ydata=(1/N)*thisClltm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Clltm = np.sqrt(np.diag(pcov_Clltm))
    
    wlltmmax_path[i] = popt_Clltm[0] # value of w at the peak
    wlltmerr_path[i] = perr_Clltm[0] # error of the w

# transverse
qtttm_path = np.size(Ctttm_band,0)-2
wtttmmax_path = np.zeros(qtttm_path)
wtttmerr_path = np.zeros(qtttm_path)

for i in range(qtttm_path):
    thisCtttm = Ctttm_band[i+1,:]
    popt_Ctttm,pcov_Ctttm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wtm_range,ydata=(1/N)*thisCtttm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Ctttm = np.sqrt(np.diag(pcov_Ctttm))
    
    wtttmmax_path[i] = popt_Ctttm[0] # value of w at the peak
    wtttmerr_path[i] = perr_Ctttm[0] # error of the w
    
# real part of Clt
qltrtm_path = np.size(Cltrtm_band,0)-2
wltrtmmax_path = np.zeros(qltrtm_path)
wltrtmerr_path = np.zeros(qltrtm_path)

for i in range(qltrtm_path):
    thisCltrtm = Cltrtm_band[i+1,:]
    popt_Cltrtm,pcov_Cltrtm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wtm_range,ydata=(1/N)*thisCltrtm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cltrtm = np.sqrt(np.diag(pcov_Cltrtm))
    
    wltrtmmax_path[i] = popt_Cltrtm[0] # value of w at the peak
    wltrtmerr_path[i] = perr_Cltrtm[0] # error of the w

# imaginary part of Clt
qltitm_path = np.size(Cltitm_band,0)-2
wltitmmax_path = np.zeros(qltitm_path)
wltitmerr_path = np.zeros(qltitm_path)

for i in range(qltitm_path):
    thisCltitm = Cltitm_band[i+1,:]
    popt_Cltitm,pcov_Cltitm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wtm_range,ydata=(1/N)*thisCltitm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cltitm = np.sqrt(np.diag(pcov_Cltitm))
    
    wltitmmax_path[i] = popt_Cltitm[0] # value of w at the peak
    wltitmerr_path[i] = perr_Cltitm[0] # error of the w
    

In [ ]:
# analytical calculation data
# toy model
Clltm_analytical = np.loadtxt('toy_data_k05_ka1_Cll.txt')
Ctttm_analytical = np.loadtxt('toy_data_k05_ka1_Ctt.txt')
Cltitm_analytical = np.loadtxt('toy_data_k05_ka1_Clti.txt')
Cltrtm_analytical = np.loadtxt('toy_data_k05_ka1_Cltr.txt')


### Starfish model

In [ ]:
# exclude the self-circling signal part for better fitting

Cll_noSC = copy.copy(Cllsm_band)
Cll_noSC[:,10:14]=0
Ctt_noSC = copy.copy(Cttsm_band)
Ctt_noSC[:,10:14]=0
Cltr_noSC = copy.copy(Cltrsm_band)
Cltr_noSC[:,10:14]=0
Clti_noSC = copy.copy(Cltism_band)
Clti_noSC[:,10:14]=0

In [ ]:
# fit each Cw along the path to get the max w at each q on the path
wmax_est = 35

# longitudinal 
qllsm_path = np.size(Cll_noSC,0)-3
wsm_range = wsm_band[0,:]
wllsmmax_path = np.zeros(qllsm_path)
wllsmerr_path = np.zeros(qllsm_path)

for i in range(qllsm_path):
    thisCllsm = Cll_noSC[i+1,:]
    popt_Cllsm,pcov_Cllsm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wsm_range,ydata=(1/N)*thisCllsm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cllsm = np.sqrt(np.diag(pcov_Cllsm))
    
    wllsmmax_path[i] = popt_Cllsm[0] # value of w at the peak
    wllsmerr_path[i] = perr_Cllsm[0] # error of the w

# transverse
qttsm_path = np.size(Ctt_noSC,0)-2
wttsmmax_path = np.zeros(qttsm_path)
wttsmerr_path = np.zeros(qttsm_path)

for i in range(qttsm_path):
    thisCttsm = Ctt_noSC[i+1,:]
    popt_Cttsm,pcov_Cttsm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wsm_range,ydata=(1/N)*thisCttsm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cttsm = np.sqrt(np.diag(pcov_Cttsm))
    
    wttsmmax_path[i] = popt_Cttsm[0] # value of w at the peak
    wttsmerr_path[i] = perr_Cttsm[0] # error of the w
    
# real part of Clt
qltrsm_path = np.size(Cltr_noSC,0)-6
wltrsmmax_path = np.zeros(qltrsm_path)
wltrsmerr_path = np.zeros(qltrsm_path)

for i in range(qltrsm_path):
    thisCltrsm = Cltr_noSC[i+5,:]
    popt_Cltrsm,pcov_Cltrsm = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wsm_range,ydata=(1/N)*thisCltrsm,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cltrsm = np.sqrt(np.diag(pcov_Cltrsm))
    
    wltrsmmax_path[i] = popt_Cltrsm[0] # value of w at the peak
    wltrsmerr_path[i] = perr_Cltrsm[0] # error of the w

# imaginary part of Clt
qltism_path = np.size(Clti_noSC,0)-2
wltismmax_path = np.zeros(qltism_path)
wltismerr_path = np.zeros(qltism_path)

for i in range(qltism_path):
    thisCltism = Clti_noSC[i+1,:]
    popt_Cltism,pcov_Cltism = scipy.optimize.curve_fit(skewed_Cauchy, xdata=wsm_range,ydata=(1/N)*thisCltism,bounds=((0,-np.inf,-np.inf,-1),(wmax_est,np.inf,np.inf,1)))
    perr_Cltism = np.sqrt(np.diag(pcov_Cltism))
    
    wltismmax_path[i] = popt_Cltism[0] # value of w at the peak
    wltismerr_path[i] = perr_Cltism[0] # error of the w
    

In [ ]:
# analytical calculation data
# starfish
Cllsm_analytical = np.loadtxt('starfish_data_Fst53-7_frep785-1_Cll.txt')
Cttsm_analytical = np.loadtxt('starfish_data_Fst53-7_frep785-1_Ctt.txt')
Cltism_analytical = np.loadtxt('starfish_data_Fst53-7_frep785-1_Clti.txt')
Cltrsm_analytical = np.loadtxt('starfish_data_Fst53-7_frep785-1_Cltr.txt')


## Figure of the main paper for the dispersion result 

In [ ]:
tickloc = [0,np.size(GtoK,axis=1),np.size(GtoK,axis=1)+np.size(KtoM,axis=1)-1,np.size(GtoK,axis=1)+np.size(KtoM,axis=1)+np.size(MtoG,axis=1)-1]

Ctm_band = (1/N)*(copy.copy(Clltm_band)+copy.copy(Ctttm_band))
Csm_band = (1/N)*(copy.copy(Cll_noSC)+copy.copy(Ctt_noSC))

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(25,9))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
#fig.tight_layout()

# first plot (toy model)
img1 = ax1.contourf(qtm_band,wtm_band,Ctm_band,levels=100)

ax1.plot(qtm_band[1:-2,0],wlltmmax_path,'ro',markersize=3,label=r"$(1/N)\langle J_{L}^{*}J_{L} \rangle$")
ax1.errorbar(qtm_band[1:-2,0],wlltmmax_path,yerr=wlltmerr_path,fmt='o',color='red',linewidth=3,capsize=5,markersize=2)

ax1.plot(qtm_band[1:-1,0],wtttmmax_path,'bx',markersize=7,label=r"$(1/N)\langle J_{T}^{*}J_{T} \rangle$")
ax1.errorbar(qtm_band[1:-1,0],wtttmmax_path,yerr=wtttmerr_path,fmt='x',color='blue',linewidth=3,capsize=5,markersize=7)

ax1.plot(qtm_band[1:-1,0],wltrtmmax_path,'^',color='orange',markersize=7,label=r"Re[$(1/N)\langle J_{L}^{*}J_{T} \rangle$]")
ax1.errorbar(qtm_band[1:-1,0],wltrtmmax_path,yerr=wltrtmerr_path,fmt='^',color='orange',linewidth=2,capsize=5,markersize=7)

ax1.plot(qtm_band[1:-1,0],wltitmmax_path,'v',color='magenta',markersize=7,label=r"Im[$(1/N)\langle J_{L}^{*}J_{T} \rangle$]")
ax1.errorbar(qtm_band[1:-1,0],wltitmmax_path,yerr=wltitmerr_path,fmt='v',color='magenta',linewidth=2,capsize=5,markersize=7)

ax1.plot(Clltm_analytical[:,0],Clltm_analytical[:,1],'-',color='red',linewidth=2,label=r"$(1/N)\langle v_{L}^{*}v_{L} \rangle$")
ax1.plot(Ctttm_analytical[:,0],Ctttm_analytical[:,1],'--',color='blue',linewidth=2,label=r"$(1/N)\langle v_{T}^{*}v_{T} \rangle$")
ax1.plot(Cltrtm_analytical[1:-1,0],Cltrtm_analytical[1:-1,1],':',color='orange',linewidth=2,label=r"Re[$(1/N)\langle v_{L}^{*}v_{T} \rangle$]")
ax1.plot(Cltitm_analytical[:,0],Cltitm_analytical[:,1],'-.',color='magenta',linewidth=2,label=r"Im[$(1/N)\langle v_{L}^{*}v_{T} \rangle$]")

cb1 = fig.colorbar(img1,ax=ax1)
ax1.set_ylabel('$\omega$',fontsize=30)
cb1.set_label('C',fontsize=30)
ax1.set_ylim([0,7.8])
ax1.legend(fontsize=18,ncol=2)
ax1.text(-2.0,7.8,'(a)',fontsize=30,va='top',ha='right')
ax1.tick_params(labelsize=20)
cb1.ax.tick_params(labelsize=20)
ax1.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# second plot (starfish embryo model)
img2 = ax2.contourf(qsm_band,wsm_band,Csm_band,levels=100)

ax2.plot(qsm_band[1:-2,0],wllsmmax_path,'ro',markersize=3,label=r"$(1/N)\langle J_{L}^{*}J_{L} \rangle$")
ax2.errorbar(qsm_band[1:-2,0],wllsmmax_path,yerr=wllsmerr_path,fmt='o',color='red',linewidth=3,capsize=5,markersize=2)

ax2.plot(qsm_band[1:-1,0],wttsmmax_path,'bx',markersize=7,label=r"$(1/N)\langle J_{T}^{*}J_{T} \rangle$")
ax2.errorbar(qsm_band[1:-1,0],wttsmmax_path,yerr=wttsmerr_path,fmt='x',color='blue',linewidth=3,capsize=5,markersize=7)

ax2.plot(qsm_band[5:-1,0],wltrsmmax_path,'^',color='orange',markersize=7,label=r"Re[$(1/N)\langle J_{L}^{*}J_{T} \rangle$]")
ax2.errorbar(qsm_band[5:-1,0],wltrsmmax_path,yerr=wltrsmerr_path,fmt='^',color='orange',linewidth=2,capsize=5,markersize=7)

ax2.plot(qsm_band[1:-1,0],wltismmax_path,'v',color='magenta',markersize=7,label=r"Im[$(1/N)\langle J_{L}^{*}J_{T} \rangle$]")
ax2.errorbar(qsm_band[1:-1,0],wltismmax_path,yerr=wltismerr_path,fmt='v',color='magenta',linewidth=2,capsize=5,markersize=7)

ax2.plot(Cllsm_analytical[:,0],Cllsm_analytical[:,1],'-',color='red',linewidth=2,label=r"$(1/N)\langle v_{L}^{*}v_{L} \rangle$")
ax2.plot(Cttsm_analytical[:,0],Cttsm_analytical[:,1],'--',color='blue',linewidth=2,label=r"$(1/N)\langle v_{T}^{*}v_{T} \rangle$")
ax2.plot(Cltrsm_analytical[1:-1,0],Cltrsm_analytical[1:-1,1],':',color='orange',linewidth=2,label=r"Re[$(1/N)\langle v_{L}^{*}v_{T} \rangle$]")
ax2.plot(Cltism_analytical[:,0],Cltism_analytical[:,1],'-.',color='magenta',linewidth=2,label=r"Im[$(1/N)\langle v_{L}^{*}v_{T} \rangle$]")

cb2 = fig.colorbar(img2,ax=ax2)
ax2.set_ylabel('$\omega$',fontsize=30)
cb2.set_label('C',fontsize=30)
ax2.set_ylim([0,31])
ax2.legend(fontsize=18,ncol=2)
ax2.text(-2.5,31.0,'(b)',fontsize=30,va='top',ha='right')
ax2.tick_params(labelsize=20)
cb2.ax.tick_params(labelsize=20)
ax2.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# to avoid the weird white lines appearing in pdf file of the plot
for c in img1.collections:
    c.set_edgecolor("face")

for c in img2.collections:
    c.set_edgecolor("face")

#plt.savefig('band_comp_v3.pdf',dpi=200,bbox_inches='tight')
plt.show()


## Figure comparing the before and after cut-off the self-circling signal

In [ ]:
# comparison of before and after cut-off of the self-circling signal

tickloc = [0,np.size(GtoK,axis=1),np.size(GtoK,axis=1)+np.size(KtoM,axis=1)-1,np.size(GtoK,axis=1)+np.size(KtoM,axis=1)+np.size(MtoG,axis=1)-1]

Csc_band = (1/N)*(copy.copy(Cllsm_band)+copy.copy(Cttsm_band))
Cnsc_band = (1/N)*(copy.copy(Cll_noSC)+copy.copy(Ctt_noSC))

%matplotlib inline

fig = plt.figure(facecolor=(1,1,1),figsize=(25,9))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

# first plot (before cut-off)
img1 = ax1.contourf(qsm_band,wsm_band,Csc_band,levels=100)

cb1 = fig.colorbar(img1,ax=ax1)
ax1.set_ylabel('$\omega$',fontsize=30)
cb1.set_label('C',fontsize=30)
ax1.set_ylim([0,31])
ax1.text(-2.5,31.0,'(a)',fontsize=30,va='top',ha='right')
ax1.tick_params(labelsize=20)
cb1.ax.tick_params(labelsize=20)
ax1.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# second plot (after cut-off)
img2 = ax2.contourf(qsm_band,wsm_band,Cnsc_band,levels=100)

cb2 = fig.colorbar(img2,ax=ax2)
ax2.set_ylabel('$\omega$',fontsize=30)
cb2.set_label('C',fontsize=30)
ax2.set_ylim([0,31])
ax2.text(-2.5,31.0,'(b)',fontsize=30,va='top',ha='right')
ax2.tick_params(labelsize=20)
cb2.ax.tick_params(labelsize=20)
ax2.set_xticks(tickloc,["$\Gamma$","K","M","$\Gamma$"],fontsize=30)

# to avoid the weird white lines appearing in pdf file of the plot
for c in img1.collections:
    c.set_edgecolor("face")

for c in img2.collections:
    c.set_edgecolor("face")

#plt.savefig('band_starfish_cutcomp_v1.pdf',dpi=200,bbox_inches='tight')
plt.show()
